In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

from save_submission import save
from evaluator import evaluate
from data_loader import load_kdd_cup_urc, load_yahoo_A1, load_yahoo_A2, load_yahoo_A3, load_yahoo_A4, load_power_demand # Univariate Datasets
from data_loader import load_nasa, load_ecg, load_gesture, load_smd # Multivariate Datasets

from tensorflow import keras
from tensorflow.keras import layers
from tqdm.notebook import tqdm

from matplotlib import pyplot as plt

In [3]:
datasets = load_yahoo_A1(128)

  0%|          | 0/67 [00:00<?, ?it/s]

In [4]:
x_train, x_test, y_test = datasets['x_train'], datasets['x_test'], datasets['y_test']

In [5]:
total_scores = {'f1': [], 'pr_auc': [], 'roc_auc': []}

In [ ]:
for i in tqdm(range(len(x_train))):
    train_df = x_train[i]
    test_df = x_test[i]
    
    Bi = layers.Bidirectional
    LSTM = layers.LSTM
    model = keras.Sequential(
        [
            layers.InputLayer(input_shape=(train_df.shape[1], train_df.shape[2])),
            Bi(LSTM(64, return_sequences=True)),
            layers.Dropout(rate=0.2),
            Bi(LSTM(32)),
            layers.RepeatVector(train_df.shape[1]),
            Bi(LSTM(32, return_sequences=True)),
            layers.Dropout(rate=0.2),
            Bi(LSTM(64)),
            layers.Dense(train_df.shape[1]),
            layers.Reshape([train_df.shape[1], train_df.shape[2]])
        ]
    )
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="mse")
    history = model.fit(train_df, train_df, epochs=50, batch_size=128, validation_split=0.3, verbose=0,
                        callbacks=[keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min", restore_best_weights=True)])    
    
    
    scores = evaluate(test_df, model.predict(test_df), y_test[i], is_reconstructed=True)
    
    total_scores['f1'].append(np.max(scores['f1']))
    total_scores['pr_auc'].append(scores['pr_auc'])
    total_scores['roc_auc'].append(scores['roc_auc'])
    print(f"Best F1 {np.max(scores['f1'])} PR-AUC: {scores['pr_auc']} ROC-AUC: {scores['roc_auc']}")

  0%|          | 0/67 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

Best F1 0.6074765929338837 PR-AUC: 0.11292811635243033 ROC-AUC: 0.6490726849153547


  0%|          | 0/1000 [00:00<?, ?it/s]

Best F1 0.9999999416666693 PR-AUC: 0.9583332995911376 ROC-AUC: 0.999999991323024


...

Best F1 0.9499999453750025 PR-AUC: 0.5952380887672342 ROC-AUC: 0.9992481151390736


  0%|          | 0/1000 [00:00<?, ?it/s]

Best F1 0.999999930000003 PR-AUC: 0.7499999441666713 ROC-AUC: 0.9999999796644299


  0%|          | 0/1000 [00:00<?, ?it/s]

Best F1 0.04516128587929277 PR-AUC: 0.0021276601423792165 ROC-AUC: 0.21452701922651163


In [ ]:
'F1', np.mean(total_scores['f1']), 'PR-AUC', np.mean(total_scores['pr_auc']), 'ROC-AUC', np.mean(total_scores['roc_auc'])